In [1]:
#jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10
import luigi
from bigquery import get_client
 
class PrintNumbers(luigi.Task):
 
    def requires(self):
        return []
 
    def output(self):
        return luigi.LocalTarget("numbers_up_to_18.csv")
 
    def run(self):
        json_key = 'clients_secret.json'
        client = get_client(json_key_file=json_key, readonly=True)
#print("started querying")
        queryStatement = "SELECT  * FROM [myproject93406:BigData.Images] where int64_field_0 > 240;"
#print(queryStatement) 
        job_id, _results = client.query(queryStatement,timeout = 25)
        complete, row_count = client.check_job(job_id)
        if complete:
            results = client.get_query_rows(job_id)
        else:
            results = "Long time to process request"
        with self.output().open('w') as f:
            for r in results:
                f.write('{}\n'.format(r))

class SquaredNumbers(luigi.Task):
 
    def requires(self):
        return [PrintNumbers()]
 
    def output(self):
        return luigi.LocalTarget("squares8.csv")
 
    def run(self):
        with self.input()[0].open() as fin, self.output().open('w') as fout:
            for line in fin:
                fout.write('{}\n'.format(line[:63]))
                
#                 n = int(line.strip())
#                 out = n * n
                
                 
if __name__ == '__main__':
    luigi.run(["--local-scheduler"], main_task_cls=SquaredNumbers)
    #luigi.run()

DEBUG: Checking if SquaredNumbers() is complete
DEBUG: Checking if PrintNumbers() is complete
INFO: Informed scheduler that task   SquaredNumbers__99914b932b   has status   PENDING
INFO: Informed scheduler that task   PrintNumbers__99914b932b   has status   PENDING
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 2
INFO: [pid 14288] Worker Worker(salt=934940021, workers=1, host=DESKTOP-P3PI6VV, username=sumed, pid=14288) running   PrintNumbers()
INFO: [pid 14288] Worker Worker(salt=934940021, workers=1, host=DESKTOP-P3PI6VV, username=sumed, pid=14288) done      PrintNumbers()
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   PrintNumbers__99914b932b   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 1
INFO: [pid 14288] Worker Worker(salt=934940021, workers=1, host=DESKTOP-P3PI6VV, username=sumed, pid=14288) running   SquaredNumbers()
INFO: [pi

In [ ]:
#Table ID	myproject93406:BigData.Images

In [ ]:
#import argparse

from google.cloud import bigquery
#from bigquery import get_client


def export_data_to_gcs(dataset_id, table_id, destination):
    json_key = 'clients_secret.json'
    bigquery_client = bigquery.Client()
 
    dataset_ref = bigquery_client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)

    job = bigquery_client.extract_table(table_ref, destination)

    job.result()  # Waits for job to complete

    print('Exported {}:{} to {}'.format(
        dataset_id, table_id, destination))


export_data_to_gcs('BigData','myproject93406:BigData.Image','gs://saxsa/imag')